## HF state

In [1]:
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np
import time
X = qml.PauliX
Y = qml.PauliY
Z = qml.PauliZ
I = qml.Identity

symbols = ["H","H","H","H"]
r_bohr = 5.0*1.8897259886
coordinates = np.array([[0.0,0.0, 1*r_bohr], [0.0, 0.0, 2*r_bohr], [0.0,0.0,3*r_bohr],[0.0, 0.0, 4*r_bohr]])
H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates, basis="sto-6g", method = "pyscf")

print('The original hamiltonian is', H)
electrons = 4
orbitals = 8
singles, doubles = qml.qchem.excitations(electrons, orbitals,fermionic=True)
print('Singles are',singles)
print('Doubles are',doubles)
hf_state = qchem.hf_state(electrons, qubits)
print(f"Total number of excitations = {len(singles) + len(doubles)}")
dev = qml.device("default.qubit", wires=qubits)
@qml.qnode(dev)
def circuit(hf_state, electrons, qubits, H):
    # Prepare the Hartree-Fock state
    print('Updated hf_state is', hf_state)
    qml.BasisState(hf_state, wires=range(qubits))

    return qml.expval(H)
print('HF state is', circuit(hf_state, electrons, qubits, H))


The original hamiltonian is -1.1091571486954506 * I(0) + 0.04196325905665701 * Z(0) + 6.339400653934679e-05 * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Y(4)) + 6.339400653934679e-05 * (X(0) @ Z(1) @ Z(2) @ Z(3) @ X(4)) + 0.04183466728352471 * Z(2) + 0.015433496464579644 * (Z(0) @ Z(2)) + 6.344558586881372e-05 * (Y(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6)) + 6.344558586881372e-05 * (X(2) @ Z(3) @ Z(4) @ Z(5) @ X(6)) + -0.004378038402021196 * (Z(0) @ Y(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6)) + -0.004378038402021196 * (Z(0) @ X(2) @ Z(3) @ Z(4) @ Z(5) @ X(6)) + 0.04170505593332402 * Z(4) + 0.019843364998430132 * (Z(0) @ Z(4)) + 0.04157626163217637 * Z(6) + 0.022112211956214756 * (Z(0) @ Z(6)) + 0.041963259056657085 * Z(1) + 0.06277200433828706 * (Z(0) @ Z(1)) + 0.0025025640470285686 * (Y(0) @ Z(2) @ Z(3) @ Y(4)) + 0.0025025640470285686 * (X(0) @ Z(2) @ Z(3) @ X(4)) + 0.04731295041972783 * (Y(0) @ X(1) @ X(2) @ Y(3)) + -0.04731295041972783 * (Y(0) @ Y(1) @ X(2) @ X(3)) + -0.04731295041972783 * (X(0) @ X(1) @ Y(2) @ Y(3)

/Users/s.poyyapakkam/miniconda3/envs/sri/lib/python3.13/site-packages/pennylane/ops/op_math/composite.py:209: FutureWarning: functools.partial will be a method descriptor in future Python versions; wrap it in staticmethod() if you want to preserve the old behavior
  return self._math_op(math.vstack(eigvals), axis=0)


## Gradient calculation - Commutator 

In [ ]:
from itertools import chain
X = qml.PauliX
Y = qml.PauliY
Z = qml.PauliZ
electrons = 4
orbitals = 8


singles, doubles = qml.qchem.excitations(electrons, orbitals, fermionic=True)
print(singles)
print(doubles)


#H, qubits = qchem.molecular_hamiltonian(symbols,geometry, active_electrons = 4, active_orbitals = 4,method = "pyscf")
print('The Hamiltonian is ', H)

w2 = []
w1 = []
e= []
o = []

max_value1 = float('-inf')
max_operator1 = None

max_value2 = float('-inf')
max_operator2 = None
print('HF state is', hf_state)
for i in singles:
    print('The current excitation operator is', i)
    w1 = qml.fermi.jordan_wigner(i)
    dev = qml.device("default.qubit", wires=qubits)
    @qml.qnode(dev)
    def commutator(H,w1, hf_state):
        qml.BasisState(hf_state, wires=range(qubits))
        res = qml.commutator(H, w1)
        return qml.expval(res)
    current_value1 = abs(commutator(H, w1, hf_state))
    print(f'The expectation value of {i} is', current_value1)

    if current_value1 > max_value1:
        max_value1 = current_value1
        max_operator1 = i
print(f"The highest single operator value is {max_value1} for operator {max_operator1}")
for j in doubles:
    print('The current double excitation operator is', j)
    w2 = qml.fermi.jordan_wigner(j)
    dev = qml.device("default.qubit", wires=qubits)
    @qml.qnode(dev)
    def commutator(H,w2, hf_state):
        qml.BasisState(hf_state, wires=range(qubits))
        res = qml.commutator(H, w2)
        return qml.expval(res)
    current_value2 = abs(commutator(H, w2,hf_state))
    print(f'The expectation value of {j} is', commutator(H, w2,hf_state))

    if current_value2 > max_value2:
        max_value2 = current_value2
        max_operator2 = j
print(f"The highest double excitation operator value is {max_value2} for operator {max_operator2}")   

if max_value2 > max_value1:
    e = max_value2
    o = max_operator2
    o_str = str(o) 
    print(f"The highest double excitation operator value is {e} for operator {o_str}")  
else:
    e = max_value1
    o = max_operator1
    o_str = str(o) 
    print(f"The highest single operator value is {e} for operator {o_str}")



## Splitting the excitation to wires 

In [ ]:
print('The operator is', o_str)
print(o_str)
ash_excitation = []

import re


# Regular expression to extract the numbers inside parentheses
indices = re.findall(r'\((\d+)\)', o_str)

# Convert the extracted strings to integers
excitations= [int(index) for index in indices]

# Print the result
print(excitations)

print('The length of the excitation is',len(excitations))

ash_excitation.append(excitations)
print('The length of ash excitation is',len(ash_excitation))
print('The no of qubits are', qubits)

#The order they have is p,q,r,s = [2,3,6,7]
# wires 1 = [s,r] = [7,6] = occupied orbitals
# wires 2 = [q,p] = [3,2] = unoccupied orbitals

#Updated order they have is p,q,r,s = [7,6,3,2]
# wires 1 = [s,r] = [2,3] = occupied orbitals
# wires 2 = [q,p] = [6,7] = unoccupied orbitals

wires1 = []
wires2 = []
wires = []
if len(excitations) == 4:
    wires2 = excitations[:2][::-1]  # Reverse the first two elements [7, 6] → [6, 7]
    wires1 = excitations[2:][::-1]  
else:
    wires = excitations     


print("wires2:", wires2)
print("wires1:", wires1)
print("wires are", wires)

#single excitation
# indices 𝑟 and 𝑝 run over the occupied and unoccupied molecular orbitals, 
# r = occupied p = unoccupied
# wires template = [r,p] = [occupied, unoccupied] = [4,0]
# when changed to original : [occ, unocc] = [0,4]





In [ ]:
print(ash_excitation)
print(wires1)
print(wires2)

In [ ]:
optimizer = qml.GradientDescentOptimizer(stepsize=0.5)
params = np.zeros(len(ash_excitation), requires_grad = True)
print('The parameters that are going in ', params)
fparams = []


    
dev = qml.device("default.qubit", wires=qubits)
@qml.qnode(dev)
def circuit(params, excitations, hf_state, qubits, H, wires1, wires2, wires):
    qml.BasisState(hf_state, wires=range(qubits))
    for i, excitation in enumerate(excitations):
        if len(excitations) == 4:
            #wires1 =  wires1#Occupied orbitals
            #wires2 = wires2 #Unoccupied orbitals
            qml.FermionicDoubleExcitation(weight = params[i], wires1=wires1, wires2=wires2)
            return qml.expval(H)
        else:
            qml.BasisState(hf_state, wires=range(qubits))
            qml.FermionicSingleExcitation(weight= params[i], wires = excitations)
            return qml.expval(H)

cost_fn = qml.QNode(circuit, dev,interface="autograd", diff_method="adjoint")
for n in range(100):
    print(f'Each step,the iteration is {n} and the parameter is {params}')
    params, energy = optimizer.step_and_cost(cost_fn, params,excitations=excitations, hf_state=hf_state, qubits=qubits, 
                                             H=H, wires1=wires1,wires2=wires2, wires=wires )
    if n % 1 == 0:
        print("step = {:},  E = {:.8f} Ha".format(n, energy))
fparams.append(params)

    

## So out of the first excitation 

In [ ]:
print('The parameters are', params)
print('Excitation which went in is', ash_excitation)
print('Wires 1, the occupied orbitals are', wires1)
print('Wires 2, the unoccupied orbitals are', wires2)
print(excitations)

## Making the next state

In [ ]:
import pennylane as qml

#w1 = qml.fermi.jordan_wigner(i)
print('Hartree fock state',hf_state)
print('Excitation that is going to be added', excitations)
print('Parameters that are going in', params)

dev = qml.device("default.qubit", wires=qubits)
@qml.qnode(dev)
def summa(hf_state, excitations, wires1, wires2, params):
        for i, excitation in enumerate(excitations):
                if len(excitations) == 4:
                        print('The double excitation going in ', excitations)
                        [qml.PauliX(i) for i in np.nonzero(hf_state)[0]] #HF state
                        print('The params that is going in ', params[i])
                        qml.FermionicDoubleExcitation(weight = params[i], wires1=wires1, wires2=wires2)
                        return qml.state()
                #else:
                 #Add single excitation        
ostate = summa(hf_state, excitations, wires1, wires2, params)

dev = qml.device("default.qubit", wires=qubits)
@qml.qnode(dev)
def summa1(ostate):
        qml.StatePrep(ostate, wires=range(qubits))
        return qml.expval(H)


print('After applying double excitation operator, the circuit is:\n')
print(qml.draw(summa, level="device", max_length=100)(hf_state, excitations, wires1, wires2, params))

print('Energy after applying the excitation is', summa1(ostate))


## After adding the excitation, the state is called ostate

## Going to do gradient calculation again

In [ ]:
from itertools import chain
X = qml.PauliX
Y = qml.PauliY
Z = qml.PauliZ
electrons = 4
orbitals = 8


singles, doubles = qml.qchem.excitations(electrons, orbitals, fermionic=True)
print(singles)
print(doubles)




w2 = []
w1 = []
e= []
o = []

max_value1 = float('-inf')
max_operator1 = None

max_value2 = float('-inf')
max_operator2 = None

print('After adding excitation state  is', ostate)
for i in singles:
    print('The current excitation operator is', i)
    w1 = qml.fermi.jordan_wigner(i)
    dev = qml.device("default.qubit", wires=qubits)
    @qml.qnode(dev)
    def commutator(H,w1, ostate):
        qml.StatePrep(ostate, wires=range(qubits))
        res = qml.commutator(H, w1)
        return qml.expval(res)
    current_value1 = abs(commutator(H, w1, ostate))
    print(f'The expectation value of {i} is', current_value1)

    if current_value1 > max_value1:
        max_value1 = current_value1
        max_operator1 = i
print(f"The highest single operator value is {max_value1} for operator {max_operator1}")
for j in doubles:
    print('The current double excitation operator is', j)
    w2 = qml.fermi.jordan_wigner(j)
    dev = qml.device("default.qubit", wires=qubits)
    @qml.qnode(dev)
    def commutator(H,w2, ostate):
        qml.StatePrep(ostate, wires=range(qubits))
        res = qml.commutator(H, w2)
        return qml.expval(res)
    current_value2 = abs(commutator(H, w2,ostate))
    print(f'The expectation value of {j} is', commutator(H, w2,ostate))

    if current_value2 > max_value2:
        max_value2 = current_value2
        max_operator2 = j
print(f"The highest double excitation operator value is {max_value2} for operator {max_operator2}")   

if max_value2 > max_value1:
    e1 = max_value2
    o1 = max_operator2
    o_str1 = str(o1) 
    print(f"The highest double excitation operator value is {e1} for operator {o_str1}")  
else:
    e1 = max_value1
    o1 = max_operator1
    o_str1 = str(o1) 
    print(f"The highest single operator value is {e1} for operator {o_str1}")

#dummy = qml.qchem.excitations_to_wires(e)
#print(qml.draw_mpl(commutator)(H,w2,hf_state)) 

# Original a⁺(0) a⁺(1) a(4) a(5)
#0.15496366291127073 for operator a⁺(7) a⁺(6) a(3) a(2) - Mine
#Original 0.15496366291127073 for operator a⁺(2) a⁺(3) a(6) a(7)

In [ ]:
print('The operator is', o_str1)
print(o_str1)


import re

# Regular expression to extract the numbers inside parentheses
indices = re.findall(r'\((\d+)\)', o_str1)

# Convert the extracted strings to integers
excitations1= [int(index) for index in indices]

# Print the result
print(excitations1)

print('The length of the excitation is',len(excitations1))

ash_excitation.append(excitations1)
print('The length of ash excitation is',len(ash_excitation))
print('Excitations in ash_excitation are', ash_excitation)


#The order they have is p,q,r,s = [2,3,6,7]
# wires 1 = [s,r] = [7,6] = occupied orbitals
# wires 2 = [q,p] = [3,2] = unoccupied orbitals

#Updated order they have is p,q,r,s = [7,6,3,2]
# wires 1 = [s,r] = [2,3] = occupied orbitals
# wires 2 = [q,p] = [6,7] = unoccupied orbitals

wires3 = []
wires4 = []
wires = []
if len(excitations1) == 4:
    wires4 = excitations1[:2][::-1]  # Reverse the first two elements [7, 6] → [6, 7]
    wires3 = excitations1[2:][::-1]  
else:
    wires = excitations1   


print("wires4:", wires4)
print("wires3:", wires3)
print("wires are", wires)

#single excitation
# indices 𝑟 and 𝑝 run over the occupied and unoccupied molecular orbitals, 
# r = occupied p = unoccupied
# wires template = [r,p] = [occupied, unoccupied] = [4,0]
# when changed to original : [occ, unocc] = [0,4]





## Energy calculation with ostate

In [ ]:
optimizer = qml.GradientDescentOptimizer(stepsize=0.5)
params = np.zeros(len(ash_excitation), requires_grad = True)
print('The parameters that are going in ', params)

params1 = 0.0
params2 = 0.0


#@qml.qnode(dev)
#def circuit(params, excitations1, hf_state, qubits, H, wires1, wires2, wires, wires3, wires4):
#    qml.BasisState(hf_state, wires=range(qubits))  # Prepare initial Hartree-Fock state
#    for i, excitation in enumerate(ash_excitation):
#        if len(excitation) == 4:
#            qml.FermionicDoubleExcitation(weight=params1, wires1=wires1, wires2=wires2)
#            qml.FermionicDoubleExcitation(weight=params2, wires1=wires3, wires2=wires4)
#        else:
#            qml.FermionicSingleExcitation(weight=params[i], wires=excitations1)
#    return qml.expval(H)




    
dev = qml.device("default.qubit", wires=qubits)
@qml.qnode(dev)
def circuit(params, excitations1, hf_state, qubits, H, wires1, wires2, wires, wires3, wires4):
    qml.BasisState(hf_state, wires=range(qubits))
    for i, excitations in enumerate(ash_excitation):
        if len(excitations) == 4:
            #wires1 =  wires1#Occupied orbitals
            #wires2 = wires2 #Unoccupied orbitals
            print('The present state of the excitation is',excitations)
            print('The parameters are', params[i])
            
            qml.FermionicDoubleExcitation(weight = params[i], wires1=wires1, wires2=wires2)
            print('After adding the first excitation, the params are', params[i+1])
            qml.FermionicDoubleExcitation(weight = params[i+1], wires1=wires3, wires2=wires4)
            print('After adding the second excitation, the wires are', wires1)
            return qml.expval(H)
        else:
            #Need to add single excitation properly
            #qml.StatePrep(hf_state, wires=range(qubits))
            qml.FermionicSingleExcitation(weight= params[i], wires = excitations1)
            return qml.expval(H)

cost_fn = qml.QNode(circuit, dev,interface="autograd", diff_method="adjoint")
for n in range(100):
    print(f'Each step,the iteration is {n} and the parameter is {params} and wires are {wires}')
    params, energy = optimizer.step_and_cost(cost_fn, params,excitations1=excitations1, hf_state=hf_state, qubits=qubits, 
                                             H=H, wires1=wires1,wires2=wires2, wires=wires, wires3=wires3, wires4=wires4 )
    if n % 2 == 0:
        print("step = {:},  E = {:.8f} Ha".format(n, energy))


#result = circuit(params, excitations1, hf_state, qubits, H, wires1, wires2, wires1, wires3, wires4)   

# Number of optimization steps 
#steps = 100 # Optimize the parameters 
#for step in range(steps): 
#    params, cost = optimizer.step_and_cost(cost_fn, params, excitations1, hf_state, qubits, H, wires1, wires2, wires, wires3, wires4) 
#    print(f"Step {step+1}, Cost: {cost}") # The optimized parameters print("Optimized parameters:"

In [ ]:
A5 =  -1.53547475
A3 = -1.56946035
A1 = -2.14728938

In [45]:



#ash_excitation = [[2, 6],[2, 3, 6, 7], [0, 3, 5, 6], [0, 1, 4, 5], [1, 2, 4, 7],[2, 6] ]

#ash_excitation = [[3, 7],[1, 2, 4, 7], [0, 1, 4, 5], [0, 3, 5, 6],[2, 3, 6, 7]]

ash_excitation = [[2,6],[1, 2, 4, 7],[0,1,4,5],[0, 3, 5, 6],[2, 3, 6, 7]]

print(ash_excitation)

optimizer = qml.GradientDescentOptimizer(stepsize=0.5)
#optimizer = qml.QNGOptimizer(stepsize=0.01)

params = np.zeros(len(ash_excitation), requires_grad = True)
print('The parameters that are going in ', params)
#params = np.tensor((-0.59709303, 0.96593200, -0.74237984, 0.86758707, 0.01052714, 0.01048673), requires_grad = False)
print('The parameters that are going in ', params)




dev = qml.device("default.qubit", wires=qubits)
@qml.qnode(dev)
def circuit(params, ash_excitation, hf_state, qubits, H):
    print('HF state', hf_state)
    qml.BasisState(hf_state, wires=range(qubits))
    for i, excitations in enumerate(ash_excitation):
        if len(ash_excitation[i]) == 4:
            print('The present state of the ash excitation is', ash_excitation[i])
            print('The parameters are', params[i])
            qml.FermionicDoubleExcitation(weight = params[i], wires1=ash_excitation[i][2:][::-1], wires2=ash_excitation[i][:2][::-1])
        else:
            #Need to add single excitation properly
            print('Single excitations coming in is', ash_excitation[i])
            print('Single excitations params:', params[i])
            qml.FermionicSingleExcitation(weight= params[i], wires = ash_excitation[i])
    return qml.expval(H)


cost_fn = qml.QNode(circuit, dev,interface="autograd", diff_method="adjoint")
for n in range(200):
    #print(f'Each step,the iteration is {n} and the parameter is {params}')
    params, energy = optimizer.step_and_cost(cost_fn, params, ash_excitation=ash_excitation, hf_state=hf_state, qubits=qubits,H=H)
    if n % 5 == 0:
        print("step = {:},  E = {:.8f} Ha".format(n, energy))

#print(qml.draw(circuit, level="device", max_length=100)(params, ash_excitation, hf_state, qubits, H))

print('The parameters are', params)
print('Upd energy is',energy)
#print(f'Updated energy is', {energy} and  params are {params} ')


[[2, 6], [1, 2, 4, 7], [0, 1, 4, 5], [0, 3, 5, 6], [2, 3, 6, 7]]
The parameters that are going in  [0. 0. 0. 0. 0.]
The parameters that are going in  [0. 0. 0. 0. 0.]
HF state [1 1 1 1 0 0 0 0]
Single excitations coming in is [2, 6]
Single excitations params: Autograd ArrayBox with value 0.0
The present state of the ash excitation is [1, 2, 4, 7]
The parameters are Autograd ArrayBox with value 0.0
The present state of the ash excitation is [0, 1, 4, 5]
The parameters are Autograd ArrayBox with value 0.0
The present state of the ash excitation is [0, 3, 5, 6]
The parameters are Autograd ArrayBox with value 0.0
The present state of the ash excitation is [2, 3, 6, 7]
The parameters are Autograd ArrayBox with value 0.0
step = 0,  E = -1.21602386 Ha
HF state [1 1 1 1 0 0 0 0]
Single excitations coming in is [2, 6]
Single excitations params: Autograd ArrayBox with value -3.903439933644411e-06
The present state of the ash excitation is [1, 2, 4, 7]
The parameters are Autograd ArrayBox with va

In [ ]:
energy - (-1.855435678083)
#-1.855460814655 - (-1.8554333589924752)
-1.855460814655 - (-1.8554482363123659)


-1.2578342634128603e-05

In [ ]:
print(energy, params)

## Using L-BFGS - scipy minimize

In [2]:
import numpy as np
import pennylane as qml
from scipy.optimize import minimize

# Define the Ash excitation structure (as per your example)
ash_excitation = [[2, 3, 6, 7], [0, 3, 5, 6], [0, 1, 4, 5], [1, 2, 4, 7], [1,5]]
#ash_excitation = [[2,6],[2, 3, 6, 7], [0, 3, 5, 6], [0, 1, 4, 5], [1, 2, 4, 7]]

#ash_excitation = [[0.4],[2,6],[1, 2, 4, 7],[0,1,4,5],[0, 3, 5, 6],[2, 3, 6, 7]]


dev = qml.device("default.qubit", wires=qubits)

# QNode to evaluate energy
@qml.qnode(dev)
def circuit(params, ash_excitation, hf_state, qubits, H):
    qml.BasisState(hf_state, wires=range(qubits))
    for i, excitations in enumerate(ash_excitation):
        if len(ash_excitation[i]) == 4:
            print(f'Double excitations coming in is {excitations} and parameters are {params[i]}')
            qml.FermionicDoubleExcitation(weight = params[i], wires1=ash_excitation[i][2:][::-1], wires2=ash_excitation[i][:2][::-1])
        else:
            print(f'Single excitations coming in is {excitations} and parameters are {params[i]}')
            qml.FermionicSingleExcitation(weight= params[i], wires = ash_excitation[i])
    return qml.expval(H)


#params = np.random.rand(len(ash_excitation))
# Define the cost function that returns the energy and its gradient
def cost_fn(params, ash_excitation, hf_state, qubits, H):
    # Compute the energy
    energy = circuit(params, ash_excitation, hf_state, qubits, H)
    
    # Compute the gradient of the energy with respect to the parameters
    #grad = qml.grad(circuit)(params, ash_excitation, hf_state, qubits, H)
    
    # Debugging: Print the gradient to ensure it is computed correctly
    #print("Energy:", energy)
    #print("Gradient:", grad)
    #print("Gradient shape:", grad.shape if hasattr(grad, 'shape') else "No shape attribute")
    return energy

# Initial parameters (use zeros or random values)
params_initial = np.zeros(len(ash_excitation))
bounds = [(-5, 100)]*len(ash_excitation)
#params_initial = np.random.rand(len(ash_excitation))


# Function to minimize
def minimize_function(params):
    energy= cost_fn(params, ash_excitation, hf_state, qubits, H)
    return energy

# Using scipy.optimize.minimize with L-BFGS-B
result = minimize(
    fun=minimize_function,   # The cost function
    x0=params_initial,       # Initial guess
    #jac=False,   
    bounds= bounds,             # Enable gradient computation
    method='L-BFGS-B',       # Use L-BFGS-B optimizer
    options={'disp': 1, 'maxiter': 500}  # Display options and max iterations
)

# Extract the result
optimized_params = result.x
minimized_energy = result.fun

print(f"Optimized parameters: {optimized_params}")
print(f"Minimized energy: {minimized_energy}")


Double excitations coming in is [2, 3, 6, 7] and parameters are 0.0
Double excitations coming in is [0, 3, 5, 6] and parameters are 0.0
Double excitations coming in is [0, 1, 4, 5] and parameters are 0.0
Double excitations coming in is [1, 2, 4, 7] and parameters are 0.0
Single excitations coming in is [1, 5] and parameters are 0.0
Double excitations coming in is [2, 3, 6, 7] and parameters are 1e-08
Double excitations coming in is [0, 3, 5, 6] and parameters are 0.0
Double excitations coming in is [0, 1, 4, 5] and parameters are 0.0
Double excitations coming in is [1, 2, 4, 7] and parameters are 0.0
Single excitations coming in is [1, 5] and parameters are 0.0
Double excitations coming in is [2, 3, 6, 7] and parameters are 0.0
Double excitations coming in is [0, 3, 5, 6] and parameters are 1e-08
Double excitations coming in is [0, 1, 4, 5] and parameters are 0.0
Double excitations coming in is [1, 2, 4, 7] and parameters are 0.0
Single excitations coming in is [1, 5] and parameters ar

In [ ]:
print(params)

#1.8554137389623182
 -1.8554434957257326
0.84442567 1.36603372 1.04982963 1.22694956 0.00177816 0.00176291
#-8.44368162e-01 -1.36552979e+00 -1.04955949e+00 -1.22675429e+00 -3.17765691e-05
#-8.44401862e-01 -1.36603754e+00 -1.04986704e+00 -1.22695951e+00 -2.67572595e-05
ash_excitation = [[2, 3, 6, 7], [0, 3, 5, 6], [0, 1, 4, 5], [1, 2, 4, 7],[2, 6]]
Optimized parameters: [ 8.44377225e-01  1.36602943e+00  1.04990487e+00  1.22695828e+00 -4.47552509e-05]
Minimized energy: -1.8554137512273634

ash_excitation = ash_excitation = [[2, 3, 6, 7], [0, 3, 5, 6], [0, 1, 4, 5], [0, 3, 5, 6],[2, 6]]
Optimized parameters: [ 1.56951374e+00  1.90566877e+00  2.09024562e+00 -1.22936328e+00 -2.15578222e-05]
Minimized energy: -1.8554137514270783

ash_excitation = [[2, 3, 6, 7], [0, 3, 5, 6], [0, 1, 4, 5], [0, 3, 5, 6],[3, 7]]
Optimized parameters: [ 1.56951594e+00  1.90565898e+00  2.09024890e+00 -1.22936877e+00 3.30492314e-05]
Minimized energy: -1.855413751453671

In [ ]:
Nick = -1.855435678084
GD = -1.85541375
Nick1 = -1.85546081

GD - Nick1

## Trying to use Jax

In [2]:
import jax
from jax import numpy as jnp
import pennylane as qml
import optax
import jax.numpy as jnp
from pennylane import qchem

X = qml.PauliX
Y = qml.PauliY
Z = qml.PauliZ
I = qml.Identity

symbols = ["H","H","H","H"]
r_bohr = 5.0*1.8897259886
coordinates = jnp.array([[0.0,0.0, 1*r_bohr], [0.0, 0.0, 2*r_bohr], [0.0,0.0,3*r_bohr],[0.0, 0.0, 4*r_bohr]])
H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates, basis="sto-6g", method = "pyscf")

print('The original hamiltonian is', H)
electrons = 4
orbitals = 8
singles, doubles = qml.qchem.excitations(electrons, orbitals,fermionic=True)
print('Singles are',singles)
print('Doubles are',doubles)
hf_state = qchem.hf_state(electrons, qubits)
print(f"Total number of excitations = {len(singles) + len(doubles)}")

dev = qml.device("default.qubit", wires=qubits)
#@jax.jit
@qml.qnode(dev, interface="jax")
def circuit(hf_state, electrons, qubits, H):
    # Prepare the Hartree-Fock state
    print('Updated hf_state is', hf_state)
    #qml.BasisState(hf_state, wires=range(qubits))
    [qml.PauliX(i) for i in jnp.nonzero(hf_state)[0]] #HF state

    return qml.expval(H)
print('HF state is', circuit(hf_state, electrons, qubits, H))

#dev = qml.device("default.qubit", wires=2)
ash_excitation = [[2,6],[1, 2, 4, 7],[0,1,4,5],[0, 3, 5, 6],[2, 3, 6, 7]]

#@jax.jit
#@qml.qnode(dev, interface="jax")
#def circuit(param):
#    qml.RX(param, wires=0)
#    qml.CNOT(wires=[0, 1])
#    return qml.expval(qml.PauliZ(0))
@jax.jit
@qml.qnode(dev, interface="jax")
def circuit(params, ash_excitation, hf_state, qubits, H):
    qml.BasisState(hf_state, wires=range(qubits))
    for i, excitations in enumerate(ash_excitation):
        if len(ash_excitation[i]) == 4:
            print(f'Double excitations coming in is {excitations} and parameters are {params[i]}')
            qml.FermionicDoubleExcitation(weight = params[i], wires1=ash_excitation[i][2:][::-1], wires2=ash_excitation[i][:2][::-1])
        else:
            print(f'Single excitations coming in is {excitations} and parameters are {params[i]}')
            qml.FermionicSingleExcitation(weight= params[i], wires = ash_excitation[i])
    return qml.expval(H)

optimizer = optax.adam(learning_rate=0.1)
params = jnp.zeros(len(ash_excitation))
opt_state = optimizer.init(params)

for i in range(20):
    cost, grad_circuit = jax.value_and_grad(circuit)(params, ash_excitation, hf_state, qubits, H)
    updates, opt_state = optimizer.update(grad_circuit, opt_state)
    params = optax.apply_updates(params, updates)
    print(f"step {i}, cost {cost}")

The original hamiltonian is -1.1091571486951746 * I(0) + 0.041770498014928666 * Z(0) + -2.1483201428927806e-09 * (Y(0) @ Z(1) @ Y(2)) + -2.1483201428927806e-09 * (X(0) @ Z(1) @ X(2)) + -3.49072480332637e-05 * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Y(4)) + -3.49072480332637e-05 * (X(0) @ Z(1) @ Z(2) @ Z(3) @ X(4)) + -0.00012361841302922436 * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6)) + -0.00012361841302922436 * (X(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ X(6)) + 0.04177050473714518 * Z(2) + 0.008819677628248201 * (Z(0) @ Z(2)) + -0.00012416073805267838 * (Y(2) @ Z(3) @ Y(4)) + -0.00012416073805267838 * (X(2) @ Z(3) @ X(4)) + -1.0993771308538999e-05 * (Z(0) @ Y(2) @ Z(3) @ Y(4)) + -1.0993771308538999e-05 * (Z(0) @ X(2) @ Z(3) @ X(4)) + 3.475490584664093e-05 * (Y(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6)) + 3.475490584664093e-05 * (X(2) @ Z(3) @ Z(4) @ Z(5) @ X(6)) + 2.968531249361694e-06 * (Z(0) @ Y(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6)) + 2.968531249361694e-06 * (Z(0) @ X(2) @ Z(3) @ Z(4) @ Z(5) @ X(6)) + 0.0

/opt/miniconda3/envs/penny/lib/python3.12/site-packages/jax/_src/numpy/array_methods.py:118: UserWarning: Explicitly requested dtype <class 'jax.numpy.complex128'> requested in astype is not available, and will be truncated to dtype complex64. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/jax-ml/jax#current-gotchas for more.
  return lax_numpy.astype(self, dtype, copy=copy, device=device)
/opt/miniconda3/envs/penny/lib/python3.12/site-packages/jax/_src/numpy/array_methods.py:118: UserWarning: Explicitly requested dtype <class 'jax.numpy.complex128'> requested in astype is not available, and will be truncated to dtype complex64. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/jax-ml/jax#current-gotchas for more.
  return lax_numpy.astype(self, dtype, copy=copy, device=device)
/opt/miniconda3/envs/penny/l

HF state is -0.5105748


TracerIntegerConversionError: The __index__() method was called on traced array with shape int32[]
The error occurred while tracing the function circuit at /var/folders/tg/76l0rq6n6m3bzwcmn22dk8_59wwtc3/T/ipykernel_6366/110053271.py:48 for jit. This concrete value was not available in Python because it depends on the value of the argument qubits.
See https://jax.readthedocs.io/en/latest/errors.html#jax.errors.TracerIntegerConversionError

## Trying to generate the Hamiltonian using 1 body and 2 body electronic integrals

In [ ]:
from pyscf import gto, ao2mo, scf
import numpy as np

r_bohr = 5.0*1.8897259886
mol_pyscf = gto.M(atom = '''H 0.00 0.0 9.4486299;;
                            H  0.0  0.0  18.897259886;
                            H  0.0  0.0  28.3458898;
                            H  0.0 0.0  37.794519772''', basis = 'sto-6g', unit = 'bohr')
rhf = scf.RHF(mol_pyscf)
energy = rhf.kernel()

one_ao = mol_pyscf.intor_symmetric('int1e_kin') + mol_pyscf.intor_symmetric('int1e_nuc')
two_ao = mol_pyscf.intor('int2e_sph')

#print('1 body',one_ao)
#print('2 body',two_ao)

one_mo = np.einsum('pi,pq,qj->ij', rhf.mo_coeff, one_ao, rhf.mo_coeff)
two_mo = ao2mo.incore.full(two_ao, rhf.mo_coeff)

two_mo = np.swapaxes(two_mo, 1, 3)

core_constant = np.array([rhf.energy_nuc()])
#print(core_constant)

H_fermionic = qml.qchem.fermionic_observable(core_constant, one_mo, two_mo)
#print(H_fermionic)
H = qml.jordan_wigner(H_fermionic)
print('Hamiltonian is ',H)


## Introduction of symmetry and building the Hamiltonian

In [ ]:
from pyscf import gto, ao2mo, scf
import numpy as np
import pennylane as qml

mol = gto.Mole()
mol.build(atom = f'''H 0 0 9.4486299; H  0 0 18.897259886; H 0 0 28.3458898; H 0 0 37.794519772''', symmetry='d2h', basis='sto-6g', unit='Bohr')

rhf = scf.RHF(mol)
energy = rhf.kernel()

one_ao = mol.intor_symmetric('int1e_kin') + mol.intor_symmetric('int1e_nuc')
two_ao = mol.intor('int2e_sph')

#print('1 body',one_ao)
#print('2 body',two_ao)

one_mo = np.einsum('pi,pq,qj->ij', rhf.mo_coeff, one_ao, rhf.mo_coeff)
two_mo = ao2mo.incore.full(two_ao, rhf.mo_coeff)

two_mo = np.swapaxes(two_mo, 1, 3)

core_constant = np.array([rhf.energy_nuc()])
#print(core_constant)

H_fermionic = qml.qchem.fermionic_observable(core_constant, one_mo, two_mo)
#print(H_fermionic)
Hs = qml.jordan_wigner(H_fermionic)
print('Hamiltonian is ',Hs)

In [ ]:
import numpy as np
import pennylane as qml
from scipy.optimize import minimize
from pennylane import qchem

# Define the Ash excitation structure (as per your example)
ash_excitation = [[2, 3, 6, 7], [0, 3, 5, 6], [0, 1, 4, 5], [1, 2, 4, 7], [3, 7], [0,4]]
qubits = 8
electrons = 4
hf_state = qchem.hf_state(electrons, qubits)


dev = qml.device("default.qubit", wires=qubits)

# QNode to evaluate energy
@qml.qnode(dev)
def circuit(params, ash_excitation, hf_state, qubits, Hs):
    qml.BasisState(hf_state, wires=range(qubits))
    for i, excitations in enumerate(ash_excitation):
        if len(excitations) == 4:
            qml.DoubleExcitation(params[i], wires=excitations)
        elif len(excitations) == 2:
            print(f'Single excitations coming in is {excitations} and parameters are {params[i]}')
            #qml.SingleExcitation(params[i], wires=excitations)
            qml.FermionicSingleExcitation(weight= params[i], wires = excitations)
    return qml.expval(Hs)



# Initial parameters (use zeros or random values)
params_initial = np.zeros(len(ash_excitation))
print(params_initial)


# Function to minimize
def minimize_function(params):
    energy= circuit(params, ash_excitation, hf_state, qubits, Hs)
    return energy

# Using scipy.optimize.minimize with L-BFGS-B
#result = minimize(
#    fun=minimize_function,   # The cost function
#    x0=params_initial,       # Initial guess
#    #jac=False,                 # Enable gradient computation
##    method='L-BFGS-B',       # Use L-BFGS-B optimizer
    
#    options={'disp': 1, 'maxiter': 500, 'gtol' : 1e-12, 'iprint': 101, 'maxcor':100}  # Display options and max iterations
#)

result = minimize(
    fun=minimize_function,   # The cost function
    x0=params_initial,       # Initial guess
    #jac=False,                 # Enable gradient computation
    method='Powell',       # Use L-BFGS-B optimizer
    #disp = True,
    options={'disp': 'True' , 'maxiter': 1000, 'rhobeg': 9.0, 'tol': 1e-12}  # Display options and max iterations
)

# Extract the result
optimized_params = result.x
minimized_energy = result.fun

print(f"Optimized parameters: {optimized_params}")
print(f"Minimized energy: {minimized_energy}")
#5.678704E-03 -5.679299E-03

In [ ]:
-1.85541375